In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

In [2]:
# Carregando o modelo Mistral-8x7B localmente
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", trust_remote_code=True)
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [9]:
# Criar o pipeline do Hugging Face
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000, device=0)

# Integrar o Hugging Face pipeline com o LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [12]:
codigo_fonte = r'''
using OfficeOpenXml;
using PandasNet;
using System.Data;

namespace GeradorRDCV
{
    public partial class Form1 : Form
    {
        public Form1()
        {
            InitializeComponent();
            ExcelPackage.LicenseContext = LicenseContext.NonCommercial;
        }
        string projeto, gestor, contador, entidade;
        private readonly string caminhoArquivoModelo = @"C:\Users\myron\source\repos\GeradorRDCV\planilha_modelo.xlsx";
        string filePath = "";
        DataTable prestacaoContas;

        private void button1_Click(object sender, EventArgs e)
        {
            using (OpenFileDialog openFileDialog = new OpenFileDialog())
            {
                openFileDialog.Filter = "Arquivos Excel (*.xlsx)|*.xlsx";
                openFileDialog.Title = "Selecione um arquivo Excel";
                // Verificar se o usuário escolheu um arquivo
                if (openFileDialog.ShowDialog() == DialogResult.OK)
                {
                    filePath = openFileDialog.FileName;
                    MessageBox.Show($"Arquivo selecionado: {filePath}");
                }
            }

            try
            {
                using (var package = new ExcelPackage(new System.IO.FileInfo(filePath)))
                {
                    var worksheet = package.Workbook.Worksheets[0];  // Primeira planilha
                    int rows = worksheet.Dimension.Rows;
                    int cols = worksheet.Dimension.Columns;

                    this.projeto = (string)worksheet.Cells["B1"].Value;
                    this.gestor = (string)worksheet.Cells["B2"].Value;
                    this.contador = (string)worksheet.Cells["B3"].Value;
                    this.entidade = (string)worksheet.Cells["B4"].Value;
                    var dataEmissao = worksheet.Cells["B5"].Value;

                    textBoxProjeto.Text = projeto;
                    textBoxGestor.Text = gestor;
                    textBoxContador.Text = contador;
                    textBoxEntidade.Text = entidade;
                    dateTimePickerDataEmissao.Text = dataEmissao.ToString();
                }

                prestacaoContas = ExcelHelper.LoadExcelToDataTable(filePath, skipRows: 5);
                MessageBox.Show($"Planilha carregada com {prestacaoContas.Rows.Count} linhas!");

            }
            catch (Exception ex)
            {
                MessageBox.Show("Erro ao ler o arquivo: " + ex.Message);
            }

        }

        private void Form1_Load(object sender, EventArgs e)
        {

        }

        private void buttonBaixaPlanilha_Click(object sender, EventArgs e)
        {
            SaveFileDialog saveFileDialog = new SaveFileDialog
            {
                Filter = "Excel Files (*.xlsx)|*.xlsx|All Files (*.*)|*.*",
                Title = "Salvar Planilha Modelo",
                FileName = "Modelo.xlsx"
            };

            if (saveFileDialog.ShowDialog() == DialogResult.OK)
            {
                try
                {
                    // Copia o arquivo para o local escolhido pelo usuário
                    File.Copy(caminhoArquivoModelo, saveFileDialog.FileName, overwrite: true);

                    MessageBox.Show("Planilha salva com sucesso!", "Sucesso", MessageBoxButtons.OK, MessageBoxIcon.Information);
                }
                catch (Exception ex)
                {
                    MessageBox.Show($"Erro ao salvar a planilha: {ex.Message}", "Erro", MessageBoxButtons.OK, MessageBoxIcon.Error);
                }
            }
        }

        private void buttonGeraRDCV_Click(object sender, EventArgs e)
        {
            string saveFilePath = "";
            if (prestacaoContas != null)
            {
                var ordens = prestacaoContas.AsEnumerable()
                    .Select(row => row.Field<string>("Nº DA ORDEM"))
                    .Distinct()
                    .ToArray();
                string msgOrders = "";

                foreach (var ordem in ordens)
                    msgOrders += ordem + "\n";

                using (FolderBrowserDialog folderDialog = new FolderBrowserDialog())
                {
                    folderDialog.Description = "Escolha uma pasta para salvar os arquivos";
                    folderDialog.ShowNewFolderButton = true;
                    // Exibindo a caixa de diálogo e verificando se o usuário selecionou uma pasta
                    if (folderDialog.ShowDialog() == DialogResult.OK)
                        saveFilePath = folderDialog.SelectedPath;
                }

                foreach (var ordem in ordens)
                {
                    var processedData = OrderProcessor.ProcessOrderData(prestacaoContas, ordem, entidade, projeto, gestor, contador);
                    //Console.WriteLine($"Gerando documento para a ordem {ordem}...");
                    MessageBox.Show($"Gerando documento para a ordem {ordem}...");
                    if (processedData != null)
                    {
                        if (entidade.Contains("SOCIAL"))
                        {
                            DocumentGenerator.RenderSesi(processedData, saveFilePath);
                            //DocumentGenerator.PreencherTemplate(templatePath: "template_sesi.docx", outputPath: $"{saveFilePath}\\{ordem}.docx", dados: processedData);
                        }
                        else
                        {
                            DocumentGenerator.RenderSenai(processedData, saveFilePath);
                            //DocumentGenerator.PreencherTemplate(templatePath: "template_sesi.docx", outputPath: $"{saveFilePath}\\{ordem}.docx", dados: processedData);
                        }
                    }
                }
            }
            else
            {
                MessageBox.Show("Nenhuma planilha carregada!", "Erro", MessageBoxButtons.OK, MessageBoxIcon.Error);
            }

        }
    }
}
'''

In [13]:


# Criar o template de prompt
template = """
Você é um especialista em testes de software. Abaixo está um código fonte. Gere um conjunto de casos de teste manuais para a interface do programa com base no código.

Código:
{codigo}

Crie ações e verificações de teste para:
1. Testes de entradas válidas e inválidas.
2. Verificações de bordas e erros esperados.
"""

prompt_template = PromptTemplate(
    input_variables=["codigo"],
    template=template,
)

# Criar a LLMChain (fluxo de execução com LangChain)
llm_chain = LLMChain(prompt=prompt_template, llm=llm)

# Executar o fluxo e gerar os testes manuais
resposta = llm_chain.run(codigo=codigo_fonte)
print(resposta)


Você é um especialista em testes de software. Abaixo está um código fonte. Gere um conjunto de casos de teste manuais para a interface do programa com base no código.

Código:

using OfficeOpenXml;
using PandasNet;
using System.Data;

namespace GeradorRDCV
{
    public partial class Form1 : Form
    {
        public Form1()
        {
            InitializeComponent();
            ExcelPackage.LicenseContext = LicenseContext.NonCommercial;
        }
        string projeto, gestor, contador, entidade;
        private readonly string caminhoArquivoModelo = @"C:\Users\myron\source\repos\GeradorRDCV\planilha_modelo.xlsx";
        string filePath = "";
        DataTable prestacaoContas;

        private void button1_Click(object sender, EventArgs e)
        {
            using (OpenFileDialog openFileDialog = new OpenFileDialog())
            {
                openFileDialog.Filter = "Arquivos Excel (*.xlsx)|*.xlsx";
                openFileDialog.Title = "Selecione um arquivo Excel";
     

In [7]:
# Verificar se CUDA está disponível
if torch.cuda.is_available():
    print("CUDA está disponível. Usando GPU.")
else:
    print("CUDA não está disponível. Usando CPU.")

# Certificar-se de que o modelo está na GPU
device = next(model.parameters()).device
print(f"Modelo está sendo executado no dispositivo: {device}")

CUDA está disponível. Usando GPU.
Modelo está sendo executado no dispositivo: cpu
